In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import torch

import torchvision
from torchvision.utils import make_grid
from DDPM import *
from UNet import *

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

modelPath = 'DiffusionData\w\model-w_799.pth'

ddpm = DDPM(model=UNet(numClasses=8), betas=(1e-4, 0.02), numTimesteps=400, dropoutRate=0.4, device=device, numClasses=8)

# Load individual models if we need to
ddpm.load_state_dict(torch.load(modelPath))
ddpm = ddpm.to(device)

rows = 16

ddpm.eval()
with torch.no_grad():
    generatedSamples, storedSamples = ddpm.sample(rows**2, (1, 28, 28), classifierGuidance=2, classLabels=torch.Tensor([1]).to(torch.int64).to(device))

print(generatedSamples.shape)

generatedSamples[generatedSamples < 0.25] = 0

grid = make_grid(generatedSamples, nrow=rows, normalize=True)
img = torchvision.transforms.ToPILImage()(grid)
img.show()

torch.Size([256, 1, 28, 28])
